# Practical Business Analytics Week 6 Lab - Unsupervised Machine Learning

The lab from this week looks at some unsupervised machine learning methods.

Work through the cells in this Jupyter Notebook, following the instructions in the text boxes to load and analyse the data.

Run the cell below to import the necessary libraries.

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
import matplotlib.pyplot as plt

First we will read in the data from the file provided on SurreyLearn. Run this code in the cell below:

```python
data = pd.read_csv("UCI-G.csv")
data.head()
```

In this data you can see that in some columns, there are features that are ordinal, for example in the savings column. To see this, we can list the unique values in the savings column using

```python
data['Savings'].unique()
```

For our unsupervised learning task we would like to convert this data into numeric data, that matches the ordering of the possible values in this column.

An ordering of the values could be:

UNKNOWN<lt100<100to500<500to1000<gt1000

(here lt means less than and gt means greater than).



To replace the values in the savings column with a numeric value, we can use a Python dictionary.

We can create a dictionary using the syntax:

```python
dict = {"key1": "value1","key2":"value2"} 
```

where the keys are the indexes of the dictionary, and the values are returned for the corresponding key.

We can use a dictionary to replace the ordinal data in this way:

```python
savings_replace = {"UNKNOWN":0,"lt100":1,"100to500":2,"500to1000":3,"gt1000":4}
data['Savings']=data['Savings'].map(savings_replace)
```

We have used the numbers from 0 to 4 to represent the values, as we will scale them later.

The map function is usually used to apply a function to every element in a column, but it can also be used with a dictionary to replace values - any value matching a key is replaced with the associated value in the dictionary.

Do this in the cell below, and check that the savings column has been updated.

The checking column also contains ordinal values. Use the same approach as for the savings column to replace them with numeric values.

In our data we still have some categorical features, for example the 'Purpose' column. As we did in the previous labs, we can convert these to a 'one-hot' encoding, where a single column is replaced with multiple columns, one for each possible value of the categorical varaible.

In pandas the *get_dummies()* function will do this for us:

```python
data_one_hot=pd.get_dummies(data,drop_first=True)
```

Do this in the cell below.

To see how the one-hot encoding has modified the data, we can look at the 'Housing' column as an example. Look at the original values in the housing column with:

```python
data['Housing'].unique()
```

Now we can look at the new columns created by *get_dummies()* and compare with the original encoding:

```python
display(data[['Housing']].head())
display(data_one_hot[['Housing_OWN','Housing_RENT']].head())
```

Notice that one of the values, 'FREE' is missing - this is because 'FREE' is encoded by both 'Housing_OWN' and 'Housing_RENT' being False.

Finally we can drop the target variable from the data (as we are performing unsupervised machine learning), and scale the numeric columns in the data.

```python
X = data_one_hot.drop("Status",axis=1) # Drop the target feature
```

To scale the data use:

```python
numeric = ((X.dtypes==np.int64) | (X.dtypes==np.float64 ))
numeric_columns = list(numeric.index[numeric==True]) # We want to only scale the numeric columns
min_max_scaler = preprocessing.MinMaxScaler()
X_scaled = X.copy() # Make a copy of the data
X_scaled[numeric_columns] = min_max_scaler.fit_transform(X_scaled[numeric_columns])
```

## K means clustering

K-means clustering consists of defining clusters such that the total intra-cluster variation (known as total within-cluster variation) is minimised.  There are several k-means algorithms available but the most common (Hartigan and Wong 1979) defines the total within-cluster variation as the sum of squared distances Euclidean distances between items and the corresponding centroid.

$$\textrm{within\_cluster\_variation}=W(C_k)=\sum_{x_i \in C_k} (x_i-\mu_i)^2$$

where the $x_i$ are the data points belonging to cluster $C_k$, and $\mu_i$ is the mean of the data points in cluster $C_k$, corresponding to the centre of the cluster.

The total within-cluster variation measures the “goodness of the clustering” and we want this to be as small as possible for our chosen k number of clusters:

$$\textrm{total\_within}=\sum_{k=1}^K W(C_k)$$


From the lecture, here are the basic k-mean algorithm steps:

1.	You specify the k number of clusters to be created (e.g. centers=3).
2.	Randomly select k data points (data records) from the dataset as the initial cluster centres.
3.	Assign each data point to their closest centroid, based on the Euclidean distance between the data point and the centroid
4.	For each of the k clusters, update the cluster centroid by calculating the new mean values of all the data points in the cluster.  The centroid of a kth cluster is record of data, i.e. a vector of length p containing the means of all fields for the data points in the kth cluster; p is the number of fields.
5.	Iteratively minimise the total within sum of square, i.e. iterate steps 3 and 4 until the cluster assignments stop changing or the maximum number of iterations is reached.

You can see that this algorithm works on numeric values and measures the sameness (or difference) using the Euclidean distances between distinct records from the dataset.


### Initial clustering with four clusters

To start with try clustering the scaled data using the *scikit-learn* KMeans class, with the number of clusters set to 4.

First we can create the python object representing the model, and fit it to the data. *scikit-learn* provides a *fit_predict()* method for some classes that combines fitting and predicting with some data.

```python
kmeans = KMeans(n_clusters=4,n_init='auto')
clusters = kmeans.fit_predict(X_scaled)
```

Do this below and try looking at the output in the clusters array, giving the cluster assignment of each data point. 

To be able to visualise the clustering we need to reduce the number of dimensions in the data. We can do this using PCA and taking the first two principal components in the data to use for plotting in two dimenions.

```python
pca = PCA()
pca_X = pca.fit_transform(X_scaled)
plt.scatter(pca_X[:,0],pca_X[:,1],c=clusters)
```

The kmeans object representing our data has an attribute *cluster_centers_* that contains the locations of the four cluster centres. We can see their locations by applying the same PCA transformation to them and adding them to the plot.

```python
# Plot data points
pca = PCA()
pca_X = pca.fit_transform(X_scaled)
plt.scatter(pca_X[:,0],pca_X[:,1],c=clusters)

# Add points for the four cluster centres
cluster_centres = kmeans.cluster_centers_
cs = pca.transform(cluster_centres)
plt.scatter(cs[:,0],cs[:,1],c='red',marker='x')
```

### Elbow method

k-means clustering defines clusters such that the total intra-cluster variation total_within is minimised.  The $\textrm{total\_within}$ value measures the “compactness” of the clustering and we want it to be as small as possible.

The Elbow method looks at $\textrm{total\_within}$ as a function of the number of clusters and you then choose the number of clusters, such that adding another cluster doesn’t improve $\textrm{total\_within}$.

1. Compute k-means clustering for different values of k, e.g. varying k from 1 to 10 clusters.
2. For each k, calculate total_within.
3. Plot the curve of total_within  according to the number of clusters k.
4. The location of a bend (knee) in the plot is generally considered as an indicator of the appropriate number of clusters.


We can write a loop to calculate the total_within score for a range of values of k (the number of clusters), using the *intertia* attribute of the kmeans model (which returns the total_within score).

Use this code as a template to build the loop, filling in the missing parts:

```python
scores = []
ks = list(range(2,16)) # Create a list ranging from 2 to 15
for k in ks:
    # Fit a model with k clusters and find the intertia
    # You can use scores.append() to add to the scores list
    ...
    ...
```

Then we can plot the results with:

```python
plt.plot(ks,scores)
```

### Select k using Average Silhouette Method

The average silhouette approach  measures the quality of a clustering so that it determines how well each object lies within its cluster by estimating the average distance between clusters.  A high average silhouette width indicates a good clustering.  The silhouette score is a measure of how close each point in one cluster is to points in the neighbouring clusters.

$$\textrm{silhouette}_i=s(i)=\frac{b(i)-\textrm{total\_within}(i)}{\textrm{max} (\textrm{total\_within}(i),b(i))}$$

where $\textrm{total\_within}$ is the average distance between data point $i$ and all other records in the same cluster, and $b(i)$ is the smallest average distance between $i$ and all other records in any other cluster.

We can say that $\textrm{total\_within}(i)$ is a measure of how well the record $i$ is assigned to its cluster, i.e. the smaller the better. $b(i)$ is the average dissimilarity.

The average $s(i)$ over all points of a cluster is a measure of how strongly grouped the points in the cluster are.  The $s(i)$ over the entire dataset is a measure of how appropriately the data have been clustered. If there are too many or too few clusters then some of the clusters will typically display much narrower silhouettes than the rest.

To calculate the silhouette score for a clustering, we can use the *silhouette_score()* function. This function takes two arguments, *X*, and *clusters*, where *X* are the data, and *clusters* are the cluster assignments of each data point:

```python
clusters = kmeans.fit_predict(X_scaled)
score = silhouette_score(X_scaled,clusters)
```

Use this to generate a plot of the silhouette score for different numbers of clusters.

## Hierarchical Clustering

The key operation in hierarchical agglomerative clustering is to repeatedly combine the two nearest clusters into a larger cluster:

1.	Calculate the distance between every pair of records and store it in a distance matrix.
2.	Allocate every record to its own cluster.
3.	Merging the closest pairs of points based on the distances from the distance matrix and as a result the number of clusters goes down by 1.
4.	Recompute the distance between the new cluster and the old ones and stores in a new distance matrix.
5.	Repeats steps 3 and 4 until all the clusters are merged into one single cluster.

In hierarchical clustering, you group the objects into a hierarchy similar to a tree-like diagram which is called a dendrogram. The distance of split or merge (called height) is shown on the y-axis of the dendrogram.

In *scikit-learn* you can use the *AgglomerativeClustering* class to create a hierarchical clustering model. The number of clusters is specified by the *n_clusters* argument, e.g.

```python
hc = AgglomerativeClustering(n_clusters=k)
```

Try writing Python code to apply hierarchical clustering to the data for a range of numbers of clusters from 2 to 15, and plot the silhouette score for each.

We can view the tree created by the algorithm using a function from *scipy* to plot the tree.

Use this code to plot the tree:

```python
# Fit the model
hc = AgglomerativeClustering(n_clusters=4,compute_distances=True)
hc.fit(X_scaled)
# Extract the tree
children = hc.children_
distance = hc.distances_
cs = np.arange(2, children.shape[0]+2)
linkage_matrix = np.column_stack([children,distance,cs]).astype(float)
# Plot the tree - you can change p to change the depth of tree shown
_ = dendrogram(linkage_matrix,truncate_mode="level",p=5)

Fit an agglomerative clustering model to the scaled data, choosing the number of clusters to create, and use a PCA transform to plot the clusters as we did for the K-means algorithm. Try this for different numbers of clusters.